<a href="https://colab.research.google.com/github/zbreeden/trainer-model/blob/main/Glossary_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/MyDrive/FourTwentyAnalytics/modules/trainer-model/')

ValueError: Mountpoint must be in a directory that exists

In [9]:
import os
from google.colab import drive

# Create the directory if it doesn't exist
mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
os.makedirs(mountpoint, exist_ok=True)

# Mount Google Drive
drive.mount(mountpoint)

Mounted at /MyDrive/FourTwentyAnalytics/modules/trainer-model/


In [10]:
import os

mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
contents = os.listdir(mountpoint)
print(contents)

['MyDrive', '.shortcut-targets-by-id', '.Trash-0', '.Encrypted']


In [11]:
import os

mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
mydrive_path = os.path.join(mountpoint, 'MyDrive')
contents_mydrive = os.listdir(mydrive_path)
print(contents_mydrive)

['Colab Notebooks', 'FourTwentyAnalytics', 'Getting started.pdf']


In [12]:
import os

mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
fourtwentyanalytics_path = os.path.join(mountpoint, 'MyDrive', 'FourTwentyAnalytics')
contents_fourtwentyanalytics = os.listdir(fourtwentyanalytics_path)
print(contents_fourtwentyanalytics)

['seeds', 'modules']


In [13]:
import os

mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
modules_path = os.path.join(mountpoint, 'MyDrive', 'FourTwentyAnalytics', 'modules')
contents_modules = os.listdir(modules_path)
print(contents_modules)

['trainer-model']


In [14]:
import os

mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
contents_trainer_model = os.listdir(mountpoint)
print(contents_trainer_model)

['MyDrive', '.shortcut-targets-by-id', '.Trash-0', '.Encrypted']


In [16]:
from google.colab import files
from google.colab import drive
import os

# Remount Google Drive
# Use the same mountpoint as before
mountpoint = '/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
# Ensure the directory exists before mounting
os.makedirs(mountpoint, exist_ok=True)
drive.mount(mountpoint)

# Now attempt to upload the file
uploaded = files.upload('/content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/data/merge_map.csv')

Drive already mounted at /MyDrive/FourTwentyAnalytics/modules/trainer-model/; to attempt to forcibly remount, call drive.mount("/MyDrive/FourTwentyAnalytics/modules/trainer-model/", force_remount=True).


OSError: [Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive'

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os

# Change the current working directory
target_directory = '/content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/'
os.chdir(target_directory)

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")

Current working directory: /content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model


In [36]:
import os, csv, shutil
from typing import List, Dict, Any # Corrected import
import yaml

REQUIRED_FIELDS = {"key", "term", "definition"}
LIST_FIELDS = {"examples", "see_also", "tags"}
DICT_FIELDS = {"source"}

def _non_empty(v): return v not in (None, "", [], {})
def _merge_lists(a, b): return list(dict.fromkeys((a or []) + (b or [])))
def _merge_dicts(a, b):
    out = dict(a or {})
    for k, v in (b or {}).items():
        if k not in out or not _non_empty(out[k]):
            out[k] = v
    return out

def _merge_entries(a: Dict[str, Any], b: Dict[str, Any]) -> Dict[str, Any]:
    if not a: return dict(b)
    out = dict(a)
    for k, bv in b.items():
        av = out.get(k)
        if k in LIST_FIELDS:
            out[k] = _merge_lists(av, bv)
        elif k in DICT_FIELDS:
            out[k] = _merge_dicts(av, bv)
        elif not _non_empty(av) and _non_empty(bv):
            out[k] = bv
    return out
def _load_yaml_list(path: str):
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f) or []
    if not isinstance(data, list):
        raise ValueError(f"{path} must be a YAML list")
    for i, item in enumerate(data):
        if not isinstance(item, dict):
            raise ValueError(f"{path} item {i+1} is not a mapping")
        missing = REQUIRED_FIELDS - set(item.keys())
        if missing:
            raise ValueError(f"{path} item {i+1} missing: {', '.join(sorted(missing))}")
    return data

def _write_yaml(path: str, items: List[Dict[str, Any]]):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if os.path.exists(path):
        shutil.copy2(path, path + ".bak")
    with open(path, "w", encoding="utf-8") as f:
        yaml.safe_dump(items, f, sort_keys=False, allow_unicode=True, width=1000)

def merge_glossaries(inputs: List[str], out: str = "trainer-model/seeds/glossary.yml", sort_by: str = "key", dry_run: bool = False):
    merged: Dict[str, Dict[str, Any]] = {}
    seen = 0
    for p in inputs:
        items = _load_yaml_list(p)
        for e in items:
            k = e["key"]
            merged[k] = _merge_entries(merged.get(k), e)
            seen += 1
    out_list = sorted(merged.values(), key=lambda d: (d.get(sort_by) or "").lower())
    print(f"[summary] files={len(inputs)} items_in={seen} unique={len(out_list)} deduped={seen-len(out_list)}")
    if not dry_run:
        _write_yaml(out, out_list)
        print(f"[ok] wrote {out}")
    return out_list

# Replace your existing merge_from_csv with this more robust version.
import csv, os

def merge_from_csv(csv_path: str,
                   out: str = "trainer-model/seeds/glossary.yml",
                   sort_by: str = "key",
                   dry_run: bool = False):
    """
    Reads a 2-col CSV and merges the YAML files listed in the first column.
    Accepts headers like 'The_Trainer_Artifact' and 'Append_File' (case/space/BOM/underscore/hyphen tolerant).
    Also auto-detects delimiter: , ; \t |
    """

    def _canon(s: str) -> str:
        # normalize header names (strip BOM, NBSP, spaces, dashes)
        return (s or "").replace("\ufeff", "").replace("\u00a0", " ").strip().lower().replace(" ", "_").replace("-", "_")

    # 1) Open with utf-8-sig to strip BOM; sniff delimiter.
    with open(csv_path, "r", encoding="utf-8-sig", newline="") as f:
        sample = f.read(4096)
        f.seek(0)
        try:
            dialect = csv.Sniffer().sniff(sample, delimiters=",;\t|")
        except csv.Error:
            dialect = csv.excel
        rdr = csv.DictReader(f, dialect=dialect)
        headers = rdr.fieldnames or []

        # 2) Build a lookup from canonicalized → original header
        canon_map = {_canon(h): h for h in headers}

        # 3) Find the two columns, allowing a few synonyms
        src_h = (canon_map.get("the_trainer_artifact")
                 or canon_map.get("artifact")
                 or canon_map.get("source")
                 or canon_map.get("input"))
        dst_h = (canon_map.get("append_file")
                 or canon_map.get("target")
                 or canon_map.get("out")
                 or canon_map.get("output"))

        if not src_h or not dst_h:
            raise KeyError(
                "Required CSV headers not found.\n"
                f"Seen headers: {headers}\n"
                "Looking for 'The_Trainer_Artifact' and 'Append_File' (case/space/BOM tolerant)."
            )

        # 4) Collect input YAML paths from the source column
        inputs = []
        for row in rdr:
            p = (row.get(src_h) or "").strip()
            if p:
                inputs.append(p)

    # 5) Reuse your existing merge_glossaries(...)
    return merge_glossaries(inputs, out=out, sort_by=sort_by, dry_run=dry_run)


In [23]:
base = "/MyDrive/FourTwentyAnalytics/modules/trainer-model"
csv_path = f"{base}/data/merge_map.csv"
out_path = f"{base}/seeds/glossary.yml"

In [24]:
import os
print("CSV exists:", os.path.exists(csv_path))

CSV exists: False


In [25]:
import os
from glob import glob

BASE = "/content/drive/MyDrive/FourTwentyAnalytics"  # <- adjust if needed
candidates = [
    "/content/merge_map.csv",
    "/content/drive/MyDrive/merge_map.csv",
    f"{BASE}/merge_map.csv",
]

print("Drive mounted:", os.path.exists("/content/drive/MyDrive"))
for p in candidates:
    print(p, "→", os.path.exists(p))

hits = glob("/content/drive/**/merge_map.csv", recursive=True)
print("Found in Drive:", hits[:10])  # show first 10 matches


Drive mounted: True
/content/merge_map.csv → False
/content/drive/MyDrive/merge_map.csv → False
/content/drive/MyDrive/FourTwentyAnalytics/merge_map.csv → False
Found in Drive: ['/content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/data/merge_map.csv']


In [26]:
csv_path = "/content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/data/merge_map.csv"
merge_from_csv(csv_path, out=f"{base}/seeds/glossary.yml", dry_run=False)

KeyError: 'The_Trainer_Artifact'

In [27]:
BASE = "/content/drive/MyDrive/FourTwentyAnalytics"
csv_path = f"{BASE}/modules/trainer-model/data/merge_map.csv"
out_path = f"{BASE}/modules/trainer-model/seeds/glossary.yml"


In [28]:
import os
print("CSV exists:", os.path.exists(csv_path))

CSV exists: True


In [29]:
merge_from_csv(csv_path, out=out_path, dry_run=True)

KeyError: 'The_Trainer_Artifact'

In [38]:
csv_path = f"{BASE}/modules/trainer-model/data/merge_map.csv"
merge_from_csv(csv_path, out=out_path, dry_run=True)

[summary] files=5 items_in=88 unique=64 deduped=24


[{'key': 'analyst-skills',
  'term': 'Analyst Skills (Technical, Functional, Soft)',
  'definition': 'Blend of tool proficiency (spreadsheets, SQL, viz tools, programming), functional capabilities (statistics, analytical thinking, problem-solving, probing, project management), and soft skills (collaboration, communication, storytelling, curiosity, informed intuition).\n',
  'examples': ['SQL for joins; Power BI dashboards; stakeholder workshop; narrative readout'],
  'see_also': ['sql', 'data-visualization', 'statistics', 'storytelling'],
  'tags': ['ibm', 'skills'],
  'orbit': 'Delivery & Insight',
  'source': {'course': 'IBM Data Analytics', 'module': 'Module 1 – Skills'},
  'status': 'draft'},
 {'key': 'apache-spark',
  'term': 'Apache Spark',
  'definition': 'Distributed processing engine (batch & streaming) with in-memory acceleration and APIs for SQL, Python, R, Java/Scala.\n',
  'examples': ['Process engagement stream to compute rolling WIP/throughput'],
  'see_also': ['hadoop',

In [39]:
csv_path = f"{BASE}/modules/trainer-model/data/merge_map.csv"
merge_from_csv(csv_path, out=out_path, dry_run=False)

[summary] files=5 items_in=88 unique=64 deduped=24
[ok] wrote /content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/seeds/glossary.yml


[{'key': 'analyst-skills',
  'term': 'Analyst Skills (Technical, Functional, Soft)',
  'definition': 'Blend of tool proficiency (spreadsheets, SQL, viz tools, programming), functional capabilities (statistics, analytical thinking, problem-solving, probing, project management), and soft skills (collaboration, communication, storytelling, curiosity, informed intuition).\n',
  'examples': ['SQL for joins; Power BI dashboards; stakeholder workshop; narrative readout'],
  'see_also': ['sql', 'data-visualization', 'statistics', 'storytelling'],
  'tags': ['ibm', 'skills'],
  'orbit': 'Delivery & Insight',
  'source': {'course': 'IBM Data Analytics', 'module': 'Module 1 – Skills'},
  'status': 'draft'},
 {'key': 'apache-spark',
  'term': 'Apache Spark',
  'definition': 'Distributed processing engine (batch & streaming) with in-memory acceleration and APIs for SQL, Python, R, Java/Scala.\n',
  'examples': ['Process engagement stream to compute rolling WIP/throughput'],
  'see_also': ['hadoop',

In [41]:
import yaml, collections, itertools

OUT_PATH = "/content/drive/MyDrive/FourTwentyAnalytics/modules/trainer-model/seeds/glossary.yml"  # adjust if needed

with open(OUT_PATH, encoding="utf-8") as f:
    items = yaml.safe_load(f) or []

keys = [it["key"] for it in items]
dupes = [k for k, c in collections.Counter(keys).items() if c > 1]
missing_core = [it.get("key") for it in items
                if not {"key","term","definition"} <= set(it)]

print(f"items={len(items)} unique_keys={len(set(keys))} deduped={len(keys)-len(set(keys))}")
print("duplicate_keys:", dupes)
print("missing_core_fields:", missing_core[:10])

# optional spot checks – change keys as you like
def get(k):
    return next((i for i in items if i["key"] == k), None)

for k in ["baccm", "data_ecosystem"]:
    it = get(k)
    if it:
        print(f"\n[{k}] see_also →", it.get("see_also"))
        print(f"[{k}] tags     →", it.get("tags"))


items=64 unique_keys=64 deduped=0
duplicate_keys: []
missing_core_fields: []

[baccm] see_also → ['change', 'need', 'solution', 'stakeholder', 'value', 'context', 'babok-guide']
[baccm] tags     → ['cbap', 'framework']
